# Linking drive

In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# imports

In [23]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import os

# Loading the dataset

In [81]:
ratings = pd.read_csv('/content/drive/MyDrive/recommendation system/movie recommendation/archive/ratings.csv')
movies = pd.read_csv('/content/drive/MyDrive/recommendation system/movie recommendation/archive/movies_metadata.csv')
keywords = pd.read_csv('/content/drive/MyDrive/recommendation system/movie recommendation/archive/keywords.csv')
credits = pd.read_csv('/content/drive/MyDrive/recommendation system/movie recommendation/archive/credits.csv')

<ipython-input-81-84499477014f>:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv('/content/drive/MyDrive/recommendation system/movie recommendation/archive/movies_metadata.csv')


In [82]:
# ratings = ratings.head(100000)
# movies = movies.head(100000)
# keywords = keywords.head(100000)
# credits = credits.head(100000)

# EDA

##ratings

In [83]:
ratings.head(10)

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556
5,1,1968,4.0,1425942148
6,1,2762,4.5,1425941300
7,1,2918,5.0,1425941593
8,1,2959,4.0,1425941601
9,1,4226,4.0,1425942228


In [84]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100000 non-null  int64  
 1   movieId    100000 non-null  int64  
 2   rating     100000 non-null  float64
 3   timestamp  100000 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


### Checking for null values in ratings

In [85]:
for i in ratings.columns:
  print(i, ratings[i].isnull().any())

userId False
movieId False
rating False
timestamp False


### checking for duplicate values

In [86]:
ratings[ratings.duplicated(subset = ['userId', "movieId"])].count()

,0
userId,0
movieId,0
rating,0
timestamp,0


## Removing users who have rated less than 50 movies

In [87]:
df = ratings.groupby("userId")["movieId"].count()

df = df[df >= 50]
ratings = ratings[ratings["userId"].isin(df.index)]
ratings.reset_index(drop = True, inplace = True)

In [88]:
ratings.head()

,userId,movieId,rating,timestamp
0,4,223,4.0,1042668576
1,4,415,4.0,1042667925
2,4,648,4.0,1042674800
3,4,1097,5.0,1042667925
4,4,1197,4.0,1042667956


## Droping movies which have been rated by less than 100 users

In [89]:
df = ratings.groupby("movieId")["userId"].count()
df = df[df >= 200]
ratings = ratings[ratings["movieId"].isin(df.index)]
ratings.reset_index(drop = True, inplace = True)

In [90]:
ratings

,userId,movieId,rating,timestamp
0,4,1210,2.0,1042667865
1,7,318,4.0,1486253595
2,7,2571,4.0,1486253599
3,8,1,4.0,1013443596
4,8,318,5.0,1013444101
...,...,...,...,...
2557,1012,480,4.0,856478812
2558,1012,589,4.0,856478507
2559,1012,593,5.0,856478476
2560,1012,1196,5.0,856478476


## movies

In [91]:
movies.head(10)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
5,False,NaN,60000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",NaN,949,tt0113277,en,Heat,"Obsessive master thief, Neil McCauley leads a ...",...,1995-12-15,187436818.0,170.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,A Los Angeles Crime Saga,Heat,False,7.7,1886.0
6,False,NaN,58000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",NaN,11860,tt0114319,en,Sabrina,An ugly duckling having undergone a remarkable...,...,1995-12-15,0.0,127.0,"[{'iso_639_1': 'fr', 'name': 'Français'}, {'is...",Released,You are cordially invited to the most surprisi...,Sabrina,False,6.2,141.0
7,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,45325,tt0112302,en,Tom and Huck,"A mischievous young boy, Tom Sawyer, witnesses...",...,1995-12-22,0.0,97.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,The Original Bad Boys.,Tom and Huck,False,5.4,45.0
8,False,NaN,35000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,9091,tt0114576,en,Sudden Death,International action superstar Jean Claude Van...,...,1995-12-22,64350171.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Terror goes into overtime.,Sudden Death,False,5.5,174.0
9,False,"{'id': 645, 'name': 'James Bond Collection', '...",58000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",http://www.mgm.com/view/movie/757/Goldeneye/,710,tt0113189,en,GoldenEye,James Bond must unmask the mysterious head of ...,...,1995-11-16,352194034.0,130.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,No limits. No fears. No substitutes.,GoldenEye,False,6.6,1194.0


In [92]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

### Drop irrelevant columns

In [93]:
movies.drop(columns = ["adult", "belongs_to_collection", "budget", "homepage", "imdb_id", "original_language", "original_title", "production_companies", "production_countries", "release_date",  "runtime", "spoken_languages", "status", "video"], inplace = True)

In [94]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   genres        45466 non-null  object 
 1   id            45466 non-null  object 
 2   overview      44512 non-null  object 
 3   popularity    45461 non-null  object 
 4   poster_path   45080 non-null  object 
 5   revenue       45460 non-null  float64
 6   tagline       20412 non-null  object 
 7   title         45460 non-null  object 
 8   vote_average  45460 non-null  float64
 9   vote_count    45460 non-null  float64
dtypes: float64(3), object(7)
memory usage: 3.5+ MB


### Reformating the columns to be of correct types

In [95]:
movies['id'] = pd.to_numeric(movies['id'], errors='coerce').astype('Int64')
movies['popularity'] = pd.to_numeric(movies['popularity'], errors='coerce').astype('float64')
movies["poster_path"] = movies["poster_path"].astype("string")
movies["tagline"] = movies["tagline"].astype("string")
movies["title"] = movies["title"].astype("string")
movies["vote_average"] = movies["vote_average"].astype("float64")
movies["vote_count"] = movies["vote_count"].astype("float64")

In [96]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   genres        45466 non-null  object 
 1   id            45463 non-null  Int64  
 2   overview      44512 non-null  object 
 3   popularity    45460 non-null  float64
 4   poster_path   45080 non-null  string 
 5   revenue       45460 non-null  float64
 6   tagline       20412 non-null  string 
 7   title         45460 non-null  string 
 8   vote_average  45460 non-null  float64
 9   vote_count    45460 non-null  float64
dtypes: Int64(1), float64(4), object(2), string(3)
memory usage: 3.5+ MB


### Combining overview and tagline as description

In [97]:
movies["overview"] = movies["overview"].fillna("")
movies["tagline"] = movies["tagline"].fillna("")

In [98]:
movies["description"] = movies["overview"] + " " + movies["tagline"]

### splitting genres to make a list

In [99]:
def split_genre(input):
  input = eval(input)
  genre = []
  for i in input:
    genre.append(i['name'])
  return genre

movies['genres'] = movies['genres'].apply(split_genre)
movies.head()

,genres,id,overview,popularity,poster_path,revenue,tagline,title,vote_average,vote_count,description
0,"[Animation, Comedy, Family]",862,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,373554033.0,,Toy Story,7.7,5415.0,"Led by Woody, Andy's toys live happily in his ..."
1,"[Adventure, Fantasy, Family]",8844,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,262797249.0,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,When siblings Judy and Peter discover an encha...
2,"[Romance, Comedy]",15602,A family wedding reignites the ancient feud be...,11.712900,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,0.0,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,A family wedding reignites the ancient feud be...
3,"[Comedy, Drama, Romance]",31357,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,81452156.0,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,"Cheated on, mistreated and stepped on, the wom..."
4,[Comedy],11862,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,76578911.0,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,Just when George Banks has recovered from his ...


### Renaming id to movieId

In [100]:
movies.rename(columns = {"id": "movieId"}, inplace = True)

### Removing null values

In [101]:
movies.dropna(subset = ["movieId", "title", "poster_path"], inplace = True)

In [102]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45077 entries, 0 to 45465
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   genres        45077 non-null  object 
 1   movieId       45077 non-null  Int64  
 2   overview      45077 non-null  object 
 3   popularity    45077 non-null  float64
 4   poster_path   45077 non-null  string 
 5   revenue       45077 non-null  float64
 6   tagline       45077 non-null  string 
 7   title         45077 non-null  string 
 8   vote_average  45077 non-null  float64
 9   vote_count    45077 non-null  float64
 10  description   45077 non-null  string 
dtypes: Int64(1), float64(4), object(2), string(4)
memory usage: 4.2+ MB


### Removing duplicate values

In [103]:
movies.drop_duplicates(subset=["title"],inplace = True)

In [104]:
print(movies['title'].nunique())
print(movies['movieId'].nunique())

41914
41914


In [105]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 41914 entries, 0 to 45465
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   genres        41914 non-null  object 
 1   movieId       41914 non-null  Int64  
 2   overview      41914 non-null  object 
 3   popularity    41914 non-null  float64
 4   poster_path   41914 non-null  string 
 5   revenue       41914 non-null  float64
 6   tagline       41914 non-null  string 
 7   title         41914 non-null  string 
 8   vote_average  41914 non-null  float64
 9   vote_count    41914 non-null  float64
 10  description   41914 non-null  string 
dtypes: Int64(1), float64(4), object(2), string(4)
memory usage: 3.9+ MB


## Keywords

In [106]:
keywords.head(10)

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."
5,949,"[{'id': 642, 'name': 'robbery'}, {'id': 703, '..."
6,11860,"[{'id': 90, 'name': 'paris'}, {'id': 380, 'nam..."
7,45325,[]
8,9091,"[{'id': 949, 'name': 'terrorist'}, {'id': 1562..."
9,710,"[{'id': 701, 'name': 'cuba'}, {'id': 769, 'nam..."


In [107]:
keywords.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46419 entries, 0 to 46418
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        46419 non-null  int64 
 1   keywords  46419 non-null  object
dtypes: int64(1), object(1)
memory usage: 725.4+ KB


### Rename id to movieID and convert to integer

In [108]:
keywords.rename(columns = {"id": "movieId"}, inplace = True)

In [109]:
keywords["movieId"] = keywords["movieId"].astype("Int64")

### Dropping null and duplicated values

In [110]:
keywords.dropna(subset = ["movieId"], inplace = True)

In [111]:
keywords.drop_duplicates(subset = ["movieId"], inplace = True)

In [112]:
keywords.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45432 entries, 0 to 46418
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movieId   45432 non-null  Int64 
 1   keywords  45432 non-null  object
dtypes: Int64(1), object(1)
memory usage: 1.1+ MB


### Converting keywords into a list

In [113]:
def convert_keywords(input):
  input = eval(input)
  keywords = []
  for i in input:
    keywords.append(i['name'])
  return keywords

keywords["keywords"]= keywords['keywords'].apply(convert_keywords)
keywords.head()

,movieId,keywords
0,862,"[jealousy, toy, boy, friendship, friends, riva..."
1,8844,"[board game, disappearance, based on children'..."
2,15602,"[fishing, best friend, duringcreditsstinger, o..."
3,31357,"[based on novel, interracial relationship, sin..."
4,11862,"[baby, midlife crisis, confidence, aging, daug..."


## credits

In [114]:
credits.head(10)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862
5,"[{'cast_id': 25, 'character': 'Lt. Vincent Han...","[{'credit_id': '52fe4292c3a36847f802916d', 'de...",949
6,"[{'cast_id': 1, 'character': 'Linus Larrabee',...","[{'credit_id': '52fe44959251416c75039da9', 'de...",11860
7,"[{'cast_id': 2, 'character': 'Tom Sawyer', 'cr...","[{'credit_id': '52fe46bdc3a36847f810f797', 'de...",45325
8,"[{'cast_id': 1, 'character': 'Darren Francis T...","[{'credit_id': '52fe44dbc3a36847f80ae0f1', 'de...",9091
9,"[{'cast_id': 1, 'character': 'James Bond', 'cr...","[{'credit_id': '52fe426ec3a36847f801e14b', 'de...",710


In [115]:
credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


### Rename id to movieId and convert it to int64

In [116]:
credits.rename(columns = {"id" : "movieId"}, inplace = True)

In [117]:
credits["movieId"] = credits["movieId"].astype("int64")

### extracting top 5 actors in the cast

In [118]:
def get_cast(input):
  input = eval(input)
  cast = []
  for i in input:
    cast.append(i['name'])
  return cast[:5]

credits["cast"] =  credits['cast'].apply(get_cast)
credits.head()

,cast,crew,movieId
0,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[Whitney Houston, Angela Bassett, Loretta Devi...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[Steve Martin, Diane Keaton, Martin Short, Kim...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


### Extracting director from the crew

In [119]:
def get_director(input):
  input = eval(input)
  for i in input:
    if(i['job'] == 'Director'):
      return i['name']
  return np.nan

credits["director"] =  credits['crew'].apply(get_director)
credits.head()

,cast,crew,movieId,director
0,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,John Lasseter
1,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844,Joe Johnston
2,"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602,Howard Deutch
3,"[Whitney Houston, Angela Bassett, Loretta Devi...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357,Forest Whitaker
4,"[Steve Martin, Diane Keaton, Martin Short, Kim...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862,Charles Shyer


In [120]:
credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   cast      45476 non-null  object
 1   crew      45476 non-null  object
 2   movieId   45476 non-null  int64 
 3   director  44589 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.4+ MB


### Dropping null and duplicated values

In [121]:
credits.drop_duplicates(subset = ["movieId"], inplace = True)

In [122]:
credits.dropna(subset = ["movieId"], inplace = True)

In [123]:
credits.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45432 entries, 0 to 45475
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   cast      45432 non-null  object
 1   crew      45432 non-null  object
 2   movieId   45432 non-null  int64 
 3   director  44545 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.7+ MB


## general

In [124]:
n_users = ratings["userId"].nunique()
n_movies = ratings["movieId"].nunique()
print(f"Number of unique users: {n_users}")
print(f"Number of unique movies: {n_movies}")

Number of unique users: 409
Number of unique movies: 11


In [125]:
users_mean_ratings = ratings.groupby("userId")["rating"].mean()
movies_mean_ratings = ratings.groupby("movieId")["rating"].mean()
print(f"Mean rating given by users: {users_mean_ratings.mean()}")
print(f"Mean rating of movies: {movies_mean_ratings.mean()}")

Mean rating given by users: 4.093552403179543
Mean rating of movies: 4.060757508870918


In [126]:
# fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# sns.boxplot(data=users_mean_ratings, ax=axes[0])
# axes[0].set_title("Boxplot of Mean Ratings given by each users")

# sns.countplot(x=users_mean_ratings, ax=axes[1])
# axes[1].set_title("Countplot of Mean Ratings given to each movie")
# axes[1].set_ylim(0, 5)

# # sns.boxplot(data = ratings["rating"], ax = axes[1])
# # axes[1].set_title("Boxplot of ratings")

# # plt.tight_layout()
# plt.show()

In [127]:
# fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# sns.boxplot(data=movies_mean_ratings, ax=axes[0])
# axes[0].set_title("Boxplot of Mean Ratings")

# # sns.countplot(data=movies_mean_ratings, ax=axes[1])
# # axes[1].set_title("Countplot of Mean Ratings")

# plt.tight_layout()
# plt.show()

### Calculating baysian averaage


we calculate the average rating using **Bayesian Average**, given by:

$$
\text{avg_rating} = w_i \cdot A_i + (1 - w_i) \cdot S
$$

Where:
$$
 w_i = \frac{n_i}{n_i + n_{\text{avg}}}
$$
- \( n_i \): Total number of ratings given to movie \( i \)
- \( n_avg \): Average number of ratings given to a movie
- \( A_i \): Average rating of movie \( i \)
- \( S \): Mean average rating


In [128]:
n_i = ratings.groupby("movieId")["rating"].count()
print("\nn_i: \n" , n_i)
n_avg = ratings.groupby("movieId")["rating"].count().mean()
print("\nnavg: \n",n_avg)
w = n_i / (n_i + n_avg)
print("\nw: \n", w)
A = ratings.groupby("movieId")["rating"].mean()
print("\n A: \n",A)
S = ratings["rating"].mean()
print("\nS: \n", S)
baysian_avg = w * A + (1 - w) * S
print("\n baysian_avg: \n", baysian_avg)


n_i: 
 movieId
1       222
260     222
296     258
318     255
356     259
480     226
589     223
593     243
1196    210
1210    200
2571    244
Name: rating, dtype: int64

navg: 
 232.9090909090909

w: 
 movieId
1       0.488010
260     0.488010
296     0.525556
318     0.522638
356     0.526520
480     0.492472
589     0.489133
593     0.510602
1196    0.474138
1210    0.461991
2571    0.511628
Name: rating, dtype: float64

 A: 
 movieId
1       3.864865
260     4.096847
296     4.290698
318     4.413725
356     3.926641
480     3.639381
589     3.838565
593     4.166667
1196    4.176190
1210    4.015000
2571    4.239754
Name: rating, dtype: float64

S: 
 4.068110850897736

 baysian_avg: 
 movieId
1       3.968925
260     4.082134
296     4.185093
318     4.248742
356     3.993624
480     3.856973
589     3.955833
593     4.118434
1196    4.119356
1210    4.043574
2571    4.155928
Name: rating, dtype: float64


In [129]:
baysian_avg_df = baysian_avg.reset_index()
baysian_avg_df.columns = ['movieId', 'baysian_avg']
movies = movies.merge(baysian_avg_df, on='movieId', how='left')

In [130]:
movies.head()

,genres,movieId,overview,popularity,poster_path,revenue,tagline,title,vote_average,vote_count,description,baysian_avg
0,"[Animation, Comedy, Family]",862,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,373554033.0,,Toy Story,7.7,5415.0,"Led by Woody, Andy's toys live happily in his ...",NaN
1,"[Adventure, Fantasy, Family]",8844,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,262797249.0,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,When siblings Judy and Peter discover an encha...,NaN
2,"[Romance, Comedy]",15602,A family wedding reignites the ancient feud be...,11.712900,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,0.0,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,A family wedding reignites the ancient feud be...,NaN
3,"[Comedy, Drama, Romance]",31357,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,81452156.0,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,"Cheated on, mistreated and stepped on, the wom...",NaN
4,[Comedy],11862,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,76578911.0,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,Just when George Banks has recovered from his ...,NaN


In [131]:
movies.sort_values("baysian_avg", ascending=False).head(10)

,genres,movieId,overview,popularity,poster_path,revenue,tagline,title,vote_average,vote_count,description,baysian_avg
3959,"[Drama, Thriller]",318,The Million Dollar Hotel starts with a jump fr...,4.938231,/4wxOzpCjXY2RQgS3YoHuvKdItTF.jpg,0.0,,The Million Dollar Hotel,5.9,76.0,The Million Dollar Hotel starts with a jump fr...,4.248742
6274,"[Action, Thriller, Science Fiction]",296,It's been 10 years since John Connor saved Ear...,20.818907,/lz4xYdF1n09lyiCfZWtWT44SZiG.jpg,435000000.0,The Machines Will Rise.,Terminator 3: Rise of the Machines,5.9,2177.0,It's been 10 years since John Connor saved Ear...,4.185093
3330,"[Drama, Science Fiction, Adventure, Mystery]",593,Ground control has been receiving strange tran...,11.059785,/nsGLKlwEOtqatz8yRdxOlAw5utr.jpg,0.0,,Solaris,7.7,364.0,Ground control has been receiving strange tran...,4.118434
931,"[Action, Thriller, Mystery]",260,"While on vacation in London, Canadian Richard ...",5.865697,/9v283GWj9a0AC8wwC4zriNqY1lZ.jpg,0.0,Handcuffed to the girl who double-crossed him,The 39 Steps,7.4,217.0,"While on vacation in London, Canadian Richard ...",4.082134
4922,"[Comedy, Drama, Romance]",480,From an exciting Indian wedding comes a relati...,10.625504,/2LysDwqhK0GlGtQqjPCiDeiBoqU.jpg,0.0,"A stressed father, a bride-to-be with a secret...",Monsoon Wedding,6.8,59.0,From an exciting Indian wedding comes a relati...,3.856973
0,"[Animation, Comedy, Family]",862,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,373554033.0,,Toy Story,7.7,5415.0,"Led by Woody, Andy's toys live happily in his ...",NaN
1,"[Adventure, Fantasy, Family]",8844,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,262797249.0,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,When siblings Judy and Peter discover an encha...,NaN
2,"[Romance, Comedy]",15602,A family wedding reignites the ancient feud be...,11.712900,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,0.0,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,A family wedding reignites the ancient feud be...,NaN
3,"[Comedy, Drama, Romance]",31357,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,81452156.0,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,"Cheated on, mistreated and stepped on, the wom...",NaN
4,[Comedy],11862,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,76578911.0,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,Just when George Banks has recovered from his ...,NaN


In [132]:
movies.sort_values("baysian_avg", ascending=True).head(10)

,genres,movieId,overview,popularity,poster_path,revenue,tagline,title,vote_average,vote_count,description,baysian_avg
4922,"[Comedy, Drama, Romance]",480,From an exciting Indian wedding comes a relati...,10.625504,/2LysDwqhK0GlGtQqjPCiDeiBoqU.jpg,0.0,"A stressed father, a bride-to-be with a secret...",Monsoon Wedding,6.8,59.0,From an exciting Indian wedding comes a relati...,3.856973
931,"[Action, Thriller, Mystery]",260,"While on vacation in London, Canadian Richard ...",5.865697,/9v283GWj9a0AC8wwC4zriNqY1lZ.jpg,0.0,Handcuffed to the girl who double-crossed him,The 39 Steps,7.4,217.0,"While on vacation in London, Canadian Richard ...",4.082134
3330,"[Drama, Science Fiction, Adventure, Mystery]",593,Ground control has been receiving strange tran...,11.059785,/nsGLKlwEOtqatz8yRdxOlAw5utr.jpg,0.0,,Solaris,7.7,364.0,Ground control has been receiving strange tran...,4.118434
6274,"[Action, Thriller, Science Fiction]",296,It's been 10 years since John Connor saved Ear...,20.818907,/lz4xYdF1n09lyiCfZWtWT44SZiG.jpg,435000000.0,The Machines Will Rise.,Terminator 3: Rise of the Machines,5.9,2177.0,It's been 10 years since John Connor saved Ear...,4.185093
3959,"[Drama, Thriller]",318,The Million Dollar Hotel starts with a jump fr...,4.938231,/4wxOzpCjXY2RQgS3YoHuvKdItTF.jpg,0.0,,The Million Dollar Hotel,5.9,76.0,The Million Dollar Hotel starts with a jump fr...,4.248742
0,"[Animation, Comedy, Family]",862,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,373554033.0,,Toy Story,7.7,5415.0,"Led by Woody, Andy's toys live happily in his ...",NaN
1,"[Adventure, Fantasy, Family]",8844,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,262797249.0,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,When siblings Judy and Peter discover an encha...,NaN
2,"[Romance, Comedy]",15602,A family wedding reignites the ancient feud be...,11.712900,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,0.0,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,A family wedding reignites the ancient feud be...,NaN
3,"[Comedy, Drama, Romance]",31357,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,81452156.0,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,"Cheated on, mistreated and stepped on, the wom...",NaN
4,[Comedy],11862,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,76578911.0,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,Just when George Banks has recovered from his ...,NaN


#making pivot matrix

## Merging ratings and movies df

In [133]:
movie_ratings_df = pd.merge(ratings, movies, on = "movieId")
movie_ratings_df

,userId,movieId,rating,timestamp,genres,overview,popularity,poster_path,revenue,tagline,title,vote_average,vote_count,description,baysian_avg
0,7,318,4.0,1486253595,"[Drama, Thriller]",The Million Dollar Hotel starts with a jump fr...,4.938231,/4wxOzpCjXY2RQgS3YoHuvKdItTF.jpg,0.0,,The Million Dollar Hotel,5.9,76.0,The Million Dollar Hotel starts with a jump fr...,4.248742
1,8,318,5.0,1013444101,"[Drama, Thriller]",The Million Dollar Hotel starts with a jump fr...,4.938231,/4wxOzpCjXY2RQgS3YoHuvKdItTF.jpg,0.0,,The Million Dollar Hotel,5.9,76.0,The Million Dollar Hotel starts with a jump fr...,4.248742
2,11,260,3.0,1231676984,"[Action, Thriller, Mystery]","While on vacation in London, Canadian Richard ...",5.865697,/9v283GWj9a0AC8wwC4zriNqY1lZ.jpg,0.0,Handcuffed to the girl who double-crossed him,The 39 Steps,7.4,217.0,"While on vacation in London, Canadian Richard ...",4.082134
3,11,296,4.0,1231676885,"[Action, Thriller, Science Fiction]",It's been 10 years since John Connor saved Ear...,20.818907,/lz4xYdF1n09lyiCfZWtWT44SZiG.jpg,435000000.0,The Machines Will Rise.,Terminator 3: Rise of the Machines,5.9,2177.0,It's been 10 years since John Connor saved Ear...,4.185093
4,11,318,4.5,1231678204,"[Drama, Thriller]",The Million Dollar Hotel starts with a jump fr...,4.938231,/4wxOzpCjXY2RQgS3YoHuvKdItTF.jpg,0.0,,The Million Dollar Hotel,5.9,76.0,The Million Dollar Hotel starts with a jump fr...,4.248742
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1199,1011,480,4.0,965402961,"[Comedy, Drama, Romance]",From an exciting Indian wedding comes a relati...,10.625504,/2LysDwqhK0GlGtQqjPCiDeiBoqU.jpg,0.0,"A stressed father, a bride-to-be with a secret...",Monsoon Wedding,6.8,59.0,From an exciting Indian wedding comes a relati...,3.856973
1200,1012,260,5.0,856477256,"[Action, Thriller, Mystery]","While on vacation in London, Canadian Richard ...",5.865697,/9v283GWj9a0AC8wwC4zriNqY1lZ.jpg,0.0,Handcuffed to the girl who double-crossed him,The 39 Steps,7.4,217.0,"While on vacation in London, Canadian Richard ...",4.082134
1201,1012,296,5.0,856478443,"[Action, Thriller, Science Fiction]",It's been 10 years since John Connor saved Ear...,20.818907,/lz4xYdF1n09lyiCfZWtWT44SZiG.jpg,435000000.0,The Machines Will Rise.,Terminator 3: Rise of the Machines,5.9,2177.0,It's been 10 years since John Connor saved Ear...,4.185093
1202,1012,480,4.0,856478812,"[Comedy, Drama, Romance]",From an exciting Indian wedding comes a relati...,10.625504,/2LysDwqhK0GlGtQqjPCiDeiBoqU.jpg,0.0,"A stressed father, a bride-to-be with a secret...",Monsoon Wedding,6.8,59.0,From an exciting Indian wedding comes a relati...,3.856973


In [134]:
movie_ratings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1204 entries, 0 to 1203
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   userId        1204 non-null   int64  
 1   movieId       1204 non-null   int64  
 2   rating        1204 non-null   float64
 3   timestamp     1204 non-null   int64  
 4   genres        1204 non-null   object 
 5   overview      1204 non-null   object 
 6   popularity    1204 non-null   float64
 7   poster_path   1204 non-null   string 
 8   revenue       1204 non-null   float64
 9   tagline       1204 non-null   string 
 10  title         1204 non-null   string 
 11  vote_average  1204 non-null   float64
 12  vote_count    1204 non-null   float64
 13  description   1204 non-null   string 
 14  baysian_avg   1204 non-null   float64
dtypes: float64(6), int64(3), object(2), string(4)
memory usage: 141.2+ KB


### checking for null values

In [135]:
for i in movie_ratings_df.columns:
  print(i, movie_ratings_df[i].isnull().any())

userId False
movieId False
rating False
timestamp False
genres False
overview False
popularity False
poster_path False
revenue False
tagline False
title False
vote_average False
vote_count False
description False
baysian_avg False


### checking for duplicate values

In [136]:
duplicates = movie_ratings_df[movie_ratings_df.duplicated(subset=['userId', 'title'], keep=False)]
print(duplicates)

Empty DataFrame
Columns: [userId, movieId, rating, timestamp, genres, overview, popularity, poster_path, revenue, tagline, title, vote_average, vote_count, description, baysian_avg]
Index: []


## creating the pivot table

In [141]:
pivot = movie_ratings_df.pivot_table(index='title', columns='userId', values='rating', )
pivot.fillna(0, inplace=True)
pivot

userId,7,8,11,12,15,16,20,24,27,29,...,990,992,993,997,998,1002,1003,1010,1011,1012
title,,,,,,,,,,,,,,,,,,,,,
Monsoon Wedding,0.0,0.0,3.5,0.0,0.0,4.5,4.0,4.0,0.0,0.0,...,5.0,0.0,0.0,0.0,5.0,3.0,3.0,3.0,4.0,4.0
Solaris,0.0,0.0,4.0,4.0,0.0,4.5,0.0,5.0,4.5,0.0,...,0.0,5.0,0.0,0.0,5.0,4.0,4.0,4.0,0.0,5.0
Terminator 3: Rise of the Machines,0.0,0.0,4.0,5.0,4.5,5.0,4.5,5.0,0.0,3.0,...,0.0,5.0,0.0,4.5,2.0,5.0,3.0,3.0,5.0,5.0
The 39 Steps,0.0,0.0,3.0,5.0,1.5,3.5,4.0,5.0,5.0,0.0,...,0.0,0.0,4.0,0.0,0.0,5.0,0.0,5.0,3.0,5.0
The Million Dollar Hotel,4.0,5.0,4.5,4.0,0.0,5.0,0.0,5.0,4.0,0.0,...,0.0,5.0,5.0,5.0,0.0,5.0,4.5,3.0,5.0,0.0


# making the scipy matrix
this is used for efficient storage of sparse data

In [143]:
from scipy.sparse import csr_matrix
matrix = csr_matrix(pivot)
matrix

<5x386 sparse matrix of type '<class 'numpy.float64'>'
	with 1204 stored elements in Compressed Sparse Row format>

# movie based collaborative filltering model:
## predict similar movies based on similar user ratings
this content based recommender calculates the cosine similarity between the pivot vector for given movie name and rest of the movies in the pivot table. it then returns the most similar vectors. this means if a movies A and B have both been rated highly by say users x, y, z, then upon prediction for movie A, movie B will be returned and vice versa.
the intuition behind this model is that, we try to find movies rated similarly by users. if movie A was liked by user 1 through 10, and there aslso exists a movie B, liked by 9 of these users, we should recommend movie B to the 10th user. he will most likely like it aas well.

## defining the knn model

In [144]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(n_neighbors = 11, algorithm='auto', metric="cosine")
model.fit(matrix)

NearestNeighbors(metric='cosine', n_neighbors=11)

## making recommendations:

In [145]:
def movie_collaborative_model(pivot, model, movie_title, num_pred):
  if movie_title not in pivot.index:
    return "Movie not found"

  movie_idx = pivot.index.get_loc(movie_title)
  movie_value_vector = pivot.iloc[movie_idx]
  dist, ind = model.kneighbours(movie_value_vector, n_neighbours = num_pred+1)
  recommendations = [pivot.index[i] for i in ind if i != movie_idx]
  return recommendations

In [146]:
title = movies['title'].loc(0)
recommendations = movie_collaborative_model(pivot, model, title, 10)
print(recommendations)

Movie not found


# User based collaborative filltering model
## predict similar users based on ratings
in this model we are making the predicitons based on finding similar users. the intuition is once you find 2 users who have similar tastes in movies, any movie user A likes will also be liked by user B

In [147]:
pivot_t = pivot.T
matrix_t = csr_matrix(pivot_t)

## defining the knn model

In [149]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(n_neighbors = 11, algorithm='auto', metric="cosine")
model.fit(matrix_t)

NearestNeighbors(metric='cosine', n_neighbors=11)

## making recommendations:

In [150]:
def user_collaborative_model(pivot, df, model, userId, num_pred):
  if userId not in pivot.index:
    return "User not found"

  user_idx = pivot.index.get_loc(userId)
  user_value_vector = pivot.iloc[user_idx]
  dist, ind = model.kneighbours(user_value_vector, n_neighbours = 5)
  recommendations = []
  for i in ind[0]:
    u_idx = pivot.index[i]
    user_movies = df[df["user_id"] == u_idx]
    top_movies = user_movies.sort_values(by="rating", ascending=False).head(num_pred/5)
    recommendations.extend(top_movies["title"].tolist())
  return recommendations

In [151]:
user = 23
recommendations = user_collaborative_model(pivot_t, movie_ratings_df, model, title, 10)
print(recommendations)

User not found


#content based recommendation system
use nlp to get movies with similar summary

## Summary based system
using overview and tagline of the movie to get similar predictions

### Load the spacy model

In [152]:
!pip install spacy
!python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 17.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


### Apply preprocessing on description

In [153]:
movies["description"]

,description
0,"Led by Woody, Andy's toys live happily in his ..."
1,When siblings Judy and Peter discover an encha...
2,A family wedding reignites the ancient feud be...
3,"Cheated on, mistreated and stepped on, the wom..."
4,Just when George Banks has recovered from his ...
...,...
41909,It's the year 3000 AD. The world's most danger...
41910,Rising and falling between a man and woman. Ri...
41911,An artist struggles to finish his work while a...
41912,"In a small town live two brothers, one a minis..."


In [154]:
import spacy
nlp = spacy.load("en_core_web_md")

In [155]:
def preprocess_text(text):
      if isinstance(text, str):
        doc = nlp(text)
        tokens = [token.lemma_.lower() for token in doc if not token.is_stop and token.is_alpha]
        processed_text = ' '.join(tokens)
        return processed_text
      return np.nan

In [ ]:
movies["description"] = movies["description"].apply(preprocess_text)
movies["description"]

### making a tfidf model

In [ ]:
movies["description"] = movies["description"].fillna('')

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer()
X = tv.fit_transform(movies["description"]).toarray()

### Using cosine similarity to predict similar movies

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
def get_similar_article(text, vectorizer, X, df, top_n=5):
  if isinstance(text, str):
    preprocessed_text = preprocess_text(text)
    X_text = vectorizer.transform([preprocessed_text])
    similarity_scores = cosine_similarity(X, X_text).flatten()
    top_indices = similarity_scores.argsort()[-top_n:][::-1]
    return df.iloc[top_indices], similarity_scores[top_indices], similarity_scores

In [ ]:
similar_movies, scores, similarity_scores = get_similar_article(movies["description"][1], tv, X, movies,10)
print(scores)
similar_movies

## cast based content recommendation system
using cast, director, genres and keywords to predict similar movies

### Merging movies with credits and keywords dataframe

In [ ]:
movies_credits_keywords = movies.merge(credits, on="movieId")
movies_credits_keywords = movies_credits_keywords.merge(keywords, on="movieId")

### Making a seperate column consisting of cast, directors, genre and keywords.
including director 5 times to give it more weightage

In [ ]:
def get_string_from_list(list):
  return ' '.join(list)

movies_credits_keywords["cast"] = movies_credits_keywords["cast"].apply(get_string_from_list)
movies_credits_keywords["genres"] = movies_credits_keywords["genres"].apply(get_string_from_list)
movies_credits_keywords["keywords"] = movies_credits_keywords["keywords"].apply(get_string_from_list)

movies_credits_keywords["cast_director_genre_keywords"] = movies_credits_keywords["cast"] + movies_credits_keywords["director"] * 5 + movies_credits_keywords["genres"] + movies_credits_keywords["keywords"]


In [ ]:
movies_credits_keywords["cast_director_genre_keywords"][0]

In [ ]:
movies_credits_keywords['cast_director_genre_keywords'].apply(preprocess_text)
movies_credits_keywords["cast_director_genre_keywords"][0]

### Creating a count vectorizer model


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X = cv.fit_transform(movies_credits_keywords["cast_director_genre_keywords"]).toarray()

In [ ]:
similar_movies, scores, similarity_scores = get_similar_article(movies_credits_keywords["cast_director_genre_keywords"][0],cv, X, movies_credits_keywords)
print(scores)
similar_movies

## Hybrid content recommendation system

In [ ]:
movies_credits_keywords["overall_detail"] = movies_credits_keywords["description"] + movies_credits_keywords["cast_director_genre_keywords"]*2

### Creating a tfidf vectorizer model


In [ ]:
tv_full = tfidfVectorizer()
X = tv_full.fit_transform(movies_credits_keywords["cast_director_genre_keywords"]).toarray()

In [ ]:
similar_movies, scores, similarity_scores = get_similar_article(movies_credits_keywords["overall_details"][1], tv_full, X, movies_credits_keywords)
print(scores)
similar_movies